# Utilisation d'un réseau pré-entrainé

In [1]:
# Append module location to sys.path
import sys, pathlib, os
pkg_path = pathlib.Path().cwd().parent
sys.path.append(pkg_path.as_posix())
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

## 1. Chargement du modèle VGG16

In [2]:
# Imports
import numpy as np
import pandas as pd
import pathlib

from keras.applications import VGG16
from keras.models import Model
from keras.layers import Input, Dense, Flatten
from keras.utils.image_dataset import image_dataset_from_directory, dataset_utils, ALLOWLIST_FORMATS

from yawbcc.datasets import load_barcelona_wbc

# Constants
DATA_DIR = pathlib.Path.home() / 'yawbcc_data'
INPUT_SHAPE = (256, 256, 3)
NUM_CLASSES = 8

# Load metadata
meta = load_barcelona_wbc()

# Create VGG16 model
vgg16 = VGG16(weights='imagenet', input_shape=INPUT_SHAPE, include_top=False, pooling='avg')
vgg16.trainable = False

## 2. Création du dataset à 8 classes

In [3]:
train_ds, valid_ds = image_dataset_from_directory(DATA_DIR / 'barcelona', validation_split=0.2, 
                                                  image_size=INPUT_SHAPE[:2], crop_to_aspect_ratio=True,
                                                  subset='both', batch_size=32, shuffle=True, seed=2022)

Found 17092 files belonging to 8 classes.
Using 13674 files for training.
Using 3418 files for validation.


## 3. Couche finale

La couche finale est directement copiée de [l'implémentation de VGG16 par Keras](https://github.com/keras-team/keras-applications/blob/06fbeb0f16e1304f239b2296578d1c50b15a983a/keras_applications/vgg16.py#L176-L180)

In [4]:
# Custom input block
inputs = Input(shape=INPUT_SHAPE)
x = vgg16(inputs, training=False)

# Classification block
x = Flatten(name='flatten')(x)  # Useless for polling='avg' or 'max' when instantiate VGG16
x = Dense(32, activation='relu', name='fc1')(x)
x = Dense(32, activation='relu', name='fc2')(x)
x = Dense(NUM_CLASSES, activation='softmax', name='predictions')(x)

# Create wbc classifier
wbc = Model(inputs, x, name='WBC')
wbc.summary()

Model: "WBC"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 256, 256, 3)]     0         
                                                                 
 vgg16 (Functional)          (None, 512)               14714688  
                                                                 
 flatten (Flatten)           (None, 512)               0         
                                                                 
 fc1 (Dense)                 (None, 32)                16416     
                                                                 
 fc2 (Dense)                 (None, 32)                1056      
                                                                 
 predictions (Dense)         (None, 8)                 264       
                                                                 
Total params: 14,732,424
Trainable params: 17,736
Non-trainable

## 4. Entraînement

L'entrainement a été réalisé sur une autre machine

In [5]:
wbc.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
wbc.fit(train_ds, epochs=20, validation_data=valid_ds)

Epoch 1/20
428/428 [==============================] - 187s 420ms/step - loss: 1.3933 - accuracy: 0.5335 - val_loss: 0.7613 - val_accuracy: 0.7303
Epoch 2/20
428/428 [==============================] - 174s 406ms/step - loss: 0.6730 - accuracy: 0.7704 - val_loss: 0.5873 - val_accuracy: 0.7984
Epoch 3/20
428/428 [==============================] - 175s 409ms/step - loss: 0.5708 - accuracy: 0.8026 - val_loss: 0.6092 - val_accuracy: 0.7908
Epoch 4/20
428/428 [==============================] - 177s 413ms/step - loss: 0.4993 - accuracy: 0.8300 - val_loss: 0.5063 - val_accuracy: 0.8259
Epoch 5/20
428/428 [==============================] - 177s 414ms/step - loss: 0.4377 - accuracy: 0.8502 - val_loss: 0.4609 - val_accuracy: 0.8464
Epoch 6/20
428/428 [==============================] - 176s 412ms/step - loss: 0.4296 - accuracy: 0.8504 - val_loss: 0.4590 - val_accuracy: 0.8394
Epoch 7/20
428/428 [==============================] - 178s 417ms/step - loss: 0.4091 - accuracy: 0.8572 - val_loss: 0.4665 -